<a href="https://colab.research.google.com/github/eduardojdiniz/CichyWanderers/blob/b49ac01ec04b7d391eef75631c709dfd8661830c/RDMs_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RDMs loader

### Import CichyWanderers GitHub Repository

In [1]:
!git clone https://github.com/eduardojdiniz/CichyWanderers

Cloning into 'CichyWanderers'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 104 (delta 52), reused 78 (delta 30), pack-reused 0
Receiving objects: 100% (104/104), 368.56 KiB | 1.96 MiB/s, done.
Resolving deltas: 100% (52/52), done.


### Import Data loader

In [2]:
import CichyWanderers.dataloader as dataloader

### Import AlexNet loader

In [3]:
import CichyWanderers.alexnet as alexnet 

### Download and create Cichy et al, 2014 dataset

In [4]:
_, RDM_dict, stim_dict = dataloader.create_dataset()

### Download and create AlexNet model (pretrained on ImageNet) RDMs

In [5]:
alexnet_RDM_dict = alexnet.create_alexnet_RDM_dict()

  0%|          | 0/92 [00:00<?, ?it/s]/content/CichyWanderers/alexnet.py:204: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_img = torch.tensor(resize_normalize(img).unsqueeze(0))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 92/92 [00:05<00:00, 16.98it/s]


<details>
    
<summary><font color='blue'>Click here for details about the dictionaries structure</font>
</summary>

### Dictionaries structures
    
#### `RDM_dict`
    RDM_dict: dict with keys 'MEG', 'fMRI_EVC', 'fMRI_IT'
    'MEG'     : ndarray, (16, 2, 1301, 92, 92)
        16 subjects, 2 sessions, 1301 time points (from -100 ms to 1200 ms
        wrt to stimulus onset at 0 ms), 92 conditions by 92 conditions.
        The last 2 dimensions form representational dissimilarity matrices of
        decoding accuracies, symmetric accross the diagonal, with the diagonal
        undefined (NaN).
    'fMRI_EVC': ndarray, (15, 92, 92)
        15 subjects, 92 conditions by 92 conditions.
        The last 2 dimensions form a representational dissimilarity matrix of
        spearman correlation for the EVC cortex, symmetric accross the diagonal,
        with the diagonal undefined (NaN).
    'fMRI_IT' : ndarray, (15, 92, 92)
        15 subjects, 92 conditions by 92 conditions.
        The last 2 dimensions form a representational dissimilarity matrix of
        spearman correlation for the IT cortex, symmetric accross the diagonal,
        with the diagonal undefined (NaN).
        
#### `stim_dict`        
    stim_dict: dict with keys 'category', 'human', 'face', 'animate', 'natural', 'imagepath'
    'category'  : list[str], indicating category
    'human'     : list[int], indicating membership (0=not a member, 1=member)
    'face'      : list[int], indicating membership (0=not a member, 1=member)
    'animate'   : list[int], indicating membership (0=not a member, 1=member)
    'natural'   : list[int], indicating membership (0=not a member, 1=member)
    'imagepath' : list[str], jpeg image filepath

#### `alexnet_RDM_dict`
    alexnet_RDM_dict: dict with keys 'layer_1', ..., 'layer_8'
    Each item is a ndarray, (92, 92), a representational dissimilarity matrix of
    spearman correlation of the layer activation to each pair of visual stimuli
    
</details>

In [18]:
# import
import numpy as np
from sklearn.linear_model import LinearRegression

In [8]:
MEG_RDMs = RDM_dict['MEG']
layer_1 = alexnet_RDM_dict['layer_1'] 
layer_2 = alexnet_RDM_dict['layer_2'] 

In [11]:
# average RDM across subjects and sessions
MEG_RDM_sub_averaged = np.mean(MEG_RDMs,axis=(0,1))
del MEG_RDMs

In [14]:
# set upper triangular values to NaN

dict_RDM = {}
for timepoint in range(0,700):
  # Load RDM at a given timepoint 
  # +100 as the RDMs provided are from -100ms to 1000ms after the stimulus onset
  RDM = np.array(MEG_RDM_sub_averaged[timepoint])
  # Since the matrix is symmetric we set upper triangular values to NaN
  RDM[np.triu_indices(RDM.shape[0], 1)] = np.nan
  dict_RDM['RDM_' + str(timepoint)] = RDM

layer_1[np.triu_indices(layer_1.shape[0], 1)] = np.nan
layer_2[np.triu_indices(layer_2.shape[0], 1)] = np.nan

In [16]:
####  preparing to run a simple regression model   ####

dict_RDM_vect = {}
il = np.tril_indices(92, k=-1) 

for i in range(0,700):
  lala = dict_RDM["RDM_"+ str(i)]
  dict_RDM_vect['RDM_vect_' + str(i)] = np.asarray(lala[il]).reshape(-1) 

layer_1_vect = np.asarray(layer_1[il]).reshape(-1) 
layer_2_vect = np.asarray(layer_2[il]).reshape(-1) 

In [19]:
# create arrays to save...

# the r ** 2
r_squ_l1 = []
r_squ_l2 = []

# intercept
intercept_l1 =[]
intercept_l2 =[]

# slope
slope_l1 = []
slope_l2 = []

# running the regression
x1 = layer_1_vect.reshape((-1, 1))
x2 = layer_2_vect.reshape((-1, 1))

for i in range(0,700):

  y = dict_RDM_vect["RDM_vect_"+ str(i)].reshape((-1, 1))
  
  model1 = LinearRegression()
  model2 = LinearRegression()
  # y is the MEG data, and x is the DCNN layer 1/2 matrix
  model1.fit(x1, y)
  model2.fit(x2, y)

  model1 = LinearRegression().fit(x1, y)
  model2 = LinearRegression().fit(x2, y)
  # r**2
  r_squ_l1.append(model1.score(x1, y))
  r_squ_l2.append(model2.score(x2, y))
  # intercept
  intercept_l1.append(model1.intercept_)
  intercept_l2.append(model2.intercept_)
  # slope
  slope_l1.append(model1.coef_)
  slope_l2.append(model2.coef_)

In [21]:
# get y_hat
y_hat_l1 = np.zeros(shape=(x1.shape[0],len(intercept_l1)))
y_hat_l2 = np.zeros(shape=(x2.shape[0],len(intercept_l2)))

for n in range(0,len(intercept_l1)):
  for i in range(0,x1.shape[0]):
    y_hat_l1[i,n] = intercept_l1[n] + slope_l1[n] * x1[i,0]
  for k in range(0,x2.shape[0]):
    y_hat_l2[k,n] = intercept_l2[n] + slope_l2[n] * x2[k,0]

In [29]:
# get the corrected_y_hat     # CORRECT THE ERROR
corrected_y_hat_l1 = np.zeros(shape=(x1.shape[0],len(intercept_l1)))
corrected_y_hat_l2 = np.zeros(shape=(x2.shape[0],len(intercept_l2)))


for i in range(0,len(intercept_l1)):
  y = dict_RDM_vect["RDM_vect_"+ str(i)].reshape((-1, 1))
  # for l1
  corrected_y_hat_l1[:,i] = y.squeeze() - y_hat_l1[:,i] 
  # for l2
  corrected_y_hat_l2[:,i] = y.squeeze() - y_hat_l2[:,i] 